# Scraper le prix neuf des voitures

## 1. Importer la liste des modèles


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
import sys
import os
import glob
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.scraping.scraping import init_driver
from src.scraping.scraping import accept_popup_general

In [2]:
df_model = pd.read_csv('../data/processed_data/modeles_voitures.csv')

In [3]:
df_model.head()

,marque,modele,finition,annee
0,Abarth,Abarth 124 Spider,1.4 Turbo Turismo BVA,2019.0
1,Abarth,Abarth 500,1.4 Turbo T-Jet 595,2020.0
2,Abarth,Abarth 500,1.4 Turbo T-Jet 595,2022.0
3,Abarth,Abarth 500,1.4 Turbo T-Jet 595 Pista,2018.0
4,Abarth,Abarth 500,1.4 Turbo T-Jet 595 Turismo,2020.0


In [4]:
# Check missing values
df_model.isnull().sum()

marque      1
modele      1
finition    1
annee       1
dtype: int64

In [5]:
# Supprimer missing values
df_model = df_model.dropna()

In [6]:
df_model["annee"] = df_model["annee"].astype(int)

In [ ]:
# Dimension
df_model.shape

(1613, 4)

In [8]:
# Liste des modèles et années
df = df_model[["marque", "modele", "annee"]].drop_duplicates()
df = df.sort_values(by=["marque", "modele", "annee"])
df.reset_index(drop=True, inplace=True)

In [9]:
df.head()

,marque,modele,annee
0,Abarth,Abarth 124 Spider,2019
1,Abarth,Abarth 500,2018
2,Abarth,Abarth 500,2019
3,Abarth,Abarth 500,2020
4,Abarth,Abarth 500,2022


In [10]:
print(df["marque"].unique())
print(len(df["marque"].unique()))

['Abarth' 'Alfa' 'Audi' 'BMW' 'Citroen' 'Cupra' 'DS' 'Dacia' 'Fiat' 'Ford'
 'Honda' 'Hyundai' 'Infiniti' 'Jaguar' 'Jeep' 'Kia' 'Land' 'Lexus' 'MG'
 'MINI' 'Mazda' 'Mercedes-Benz' 'Mitsubishi' 'Nissan' 'Opel' 'Peugeot'
 'Renault' 'Seat' 'Skoda' 'Smart' 'Suzuki' 'Toyota' 'Volkswagen' 'Volvo']
34


In [11]:
# Rename "marque" to match with the right name used in the caradisiac website
dict_name_marque = {'Abarth' : 'Abarth', 'Alfa' : 'Alfa Romeo', 'Audi' : 'Audi', 
                    'BMW' : 'BMW', 
                    'Citroen' : 'Citroen', 'Cupra' : 'Cupra', 
                    'DS' : 'DS', 'Dacia' : 'Dacia',
                    'Fiat' : 'Fiat', 'Ford' : 'Ford', 
                    'Honda' : 'Honda', 'Hyundai' : 'Hyundai', 
                    'Infiniti' : 'Infiniti', 
                    'Jaguar' : 'Jaguar', 'Jeep' : 'Jeep',
                    'Kia' : 'Kia', 
                    'Land' : 'Land Rover', 'Lexus' : 'Lexus', 
                    'MG' : "MG", 'MINI' : 'MINI', 'Mazda' : 'Mazda', 'Mercedes-Benz' : 'Mercedes', 'Mitsubishi' : 'Mitsubishi', 
                    'Nissan' : 'Nissan', 
                    'Opel' : 'Opel', 
                    'Peugeot' : 'Peugeot', 
                    'Renault' : 'Renault', 
                    'Seat' : 'Seat', 'Skoda' : 'Skoda', 'Smart' : 'Smart', 'Suzuki' : 'Suzuki', 
                    'Toyota' : 'Toyota', 
                    'Volkswagen' : 'Volkswagen', 'Volvo' : 'Volvo'
                    }

df['marque'] = df['marque'].replace(dict_name_marque)
print(df["marque"].unique())
print(len(df["marque"].unique()))

['Abarth' 'Alfa Romeo' 'Audi' 'BMW' 'Citroen' 'Cupra' 'DS' 'Dacia' 'Fiat'
 'Ford' 'Honda' 'Hyundai' 'Infiniti' 'Jaguar' 'Jeep' 'Kia' 'Land Rover'
 'Lexus' 'MG' 'MINI' 'Mazda' 'Mercedes' 'Mitsubishi' 'Nissan' 'Opel'
 'Peugeot' 'Renault' 'Seat' 'Skoda' 'Smart' 'Suzuki' 'Toyota' 'Volkswagen'
 'Volvo']
34


In [12]:
freq_marque = df["marque"].value_counts().reset_index().sort_values(by='marque')
freq_marque

,marque,count
25,Abarth,7
21,Alfa Romeo,10
5,Audi,45
8,BMW,35
1,Citroen,53
28,Cupra,4
19,DS,13
15,Dacia,19
12,Fiat,29
6,Ford,37


## 2. Essai avec request et BeautifulSoup

In [13]:
# Function to scrape car catalogue prices from Caradisiac
def clean_model_name(model):
    # Remove special characters and normalize spaces
    model_v2 = re.sub(r'[^\w\s]', '', model).strip().lower()
    model_v3 = model_v2.replace(' ', '-')
    return model_v3

def scrape_caradisiac_price(modele, annee):
    base_url = "https://www.caradisiac.com/fiches-techniques/modele--"
    
    # Build search URL
    clean_model = clean_model_name(modele)
    search_url = f"{base_url}{clean_model}/{str(annee)}"
    
    print(f"Search url original pattern is: {search_url}")
    
    try:
        # First attempt with original pattern
        time.sleep(5)
        response = requests.get(search_url)
        #print(f"Code Status: {response.status_code}")
        if response.status_code != 200:
            # Try alternative URL patterns
            alternative_models = [
                clean_model.replace('_', ''), # Remove "_"
                '-'.join([clean_model.replace('_', ''), "2e", "generation"])  # join with version
            ] + ['-'.join([clean_model.replace('_', ''), str(i)]) for i in range(1,6)]
            
            #print(f"Alternative urls: {alternative_models}")
            
            for alt_model in alternative_models:
                time.sleep(5)
                alt_url = f"{base_url}{alt_model}/{str(annee)}"
                response = requests.get(alt_url)
                if response.status_code == 200:
                    search_url = alt_url
                    break
        # Add delay to be respectful to the server
        time.sleep(5)
        
        # Make request
        print(f"L'url final est: {search_url}")
        response = requests.get(search_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all car versions
        versions = soup.find_all('table', class_='listingTab')

        # Extract all rows from the table
        table = versions[0]  # Since versions contains only one table
        rows = table.find_all('tr')

        # Initialize lists to store data
        data = []
        headers = []

        # Get headers from first row
        header_row = rows[0]
        headers = [th.get_text(strip=True) for th in header_row.find_all('th')]

        # Get data from remaining rows
        for row in rows[1:]:
            cols = row.find_all('td')
            row_data = [col.get_text(strip=True) for col in cols]
            if row_data:  # Only add non-empty rows
                data.append(row_data)
        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        df['url'] = search_url
        return df    
    except Exception as e:
        print(f"Error scraping {modele}: {str(e)}")
        return None

In [14]:
# # Create an empty DataFrame to store all results
# df_all = pd.DataFrame()

# # Add progress tracking
# total_models = len(df)
# print(f"Starting to scrape {total_models} models...")

# for idx, row in df.iterrows():
#     try:
#         print(f"Processing {idx+1}/{total_models}: {row['modele']} ({row['annee']})")
#         df_version = scrape_caradisiac_price(row['modele'], row['annee'])
        
#         if df_version is not None:
#             # Add model and year columns to identify the source
#             df_version.insert(0, 'source_model', row['modele'])
#             df_version.insert(1, 'source_year', row['annee'])
#             df_all = pd.concat([df_all, df_version], ignore_index=True)
            
#     except Exception as e:
#         print(f"Error processing {row['modele']} ({row['annee']}): {str(e)}")
#         continue

# print(f"\nScraping completed. Total entries collected: {len(df_all)}")

## 3. Essai avec Selenium pour retrouver les bons urls


In [15]:
def select_option_contain (select_element, partial_text):
    select = Select(select_element)

    #print(f"Partial text est: {partial_text}")
    accents = {
            'a': r'[àáâãäå]',
            'e': r'[èéêë]',
            'i': r'[ìíîï]',
            'o': r'[òóôõö]',
            'u': r'[ùúûü]',
            'c': r'[ç]',
            'n': r'[ñ]',
            'A': r'[ÀÁÂÃÄÅ]',
            'E': r'[ÈÉÊË]',
            'I': r'[ÌÍÎÏ]',
            'O': r'[ÒÓÔÕÖ]',
            'U': r'[ÙÚÛÜ]',
            'C': r'[Ç]',
            'N': r'[Ñ]',
        }
    # Remove special characters with accent
    for remplacement, pattern in accents.items():
        partial_text = re.sub(pattern, remplacement, partial_text)
    print(f"Partial text: {partial_text}")

    # First check for exact matches
    for option in select.options:
        #if option.get_attribute("value").strip() != "" and partial_text.strip().lower() == option.text.strip().lower():
        if (option.get_attribute("value").strip() != "" and 
            partial_text.lower() == option.text.lower()
            ):
            select.select_by_visible_text(option.text)
            print(f"Found exact match: {option.text}")
            option_select = "Very Exact match"
            return option.text, option_select
            break
        
    # Check for exact matches after removing space and "-"
    for option in select.options:
        if (option.get_attribute("value").strip() != "" and 
            re.sub(r"[\s\-]", "", partial_text.strip().lower()) == re.sub(r"[\s\-]", "", option.text.strip().lower())
            ): # remove any space and "-"
            select.select_by_visible_text(option.text)
            print(f"Found exact match: {option.text}")
            option_select = "Exact match"
            return option.text, option_select
            break
             
    # If no exact match found, check for partial matches
    best_match = None
    max_common_chars = 0
    
    for option in select.options:
        if option.get_attribute("value").strip() != "":
            option_text = re.sub(r"[\s\-]", "", option.text.strip().lower()) # remove any space
            partial_text_lower = re.sub(r"[\s\-]", "", partial_text.strip().lower()) # remove any space
            
            # Count common characters
            common_chars = sum(1 for c in option_text if c in partial_text_lower)
            
            # Check if this is a partial match and has more common characters
            if ((partial_text_lower in option_text) or (option_text in partial_text_lower)) and common_chars > max_common_chars:
                max_common_chars = common_chars
                best_match = option
    # Retain the best partial match = the option that has more common characters with the partial text
    if best_match is not None:
        select.select_by_visible_text(best_match.text)
        print(f"Found best partial match: {best_match.text}")
        option_select = "Partial match"
        return best_match.text, option_select


In [16]:
def clean_name(model):
    # Remove special characters and normalize spaces
    accents = {
            'a': r'[àáâãäå]',
            'e': r'[èéêë]',
            'i': r'[ìíîï]',
            'o': r'[òóôõö]',
            'u': r'[ùúûü]',
            'c': r'[ç]',
            'n': r'[ñ]',
            'A': r'[ÀÁÂÃÄÅ]',
            'E': r'[ÈÉÊË]',
            'I': r'[ÌÍÎÏ]',
            'O': r'[ÒÓÔÕÖ]',
            'U': r'[ÙÚÛÜ]',
            'C': r'[Ç]',
            'N': r'[Ñ]',
        }

    for remplacement, pattern in accents.items():
        model = re.sub(pattern, remplacement, model)
    model = re.sub(r'[^\w\s\-!]', '', model).strip().lower()

    model = model.replace(' ', '-')
    
    return model

In [17]:
# list_models = pd.unique(df['modele'])
# print(list_models)
# print([i for i in list_models if "!" in i ])

# list_models_cleaned = [clean_name(model) for model in list_models]
# print(list_models_cleaned)

# list_models_test = [any(char in i for char in 'éèêëàâäôöûüçñ') for i in list_models_cleaned]
# print([i for i in list_models_test if i == True])

In [18]:
def collect_prix_neuf (url, marque, modele, annee, wait_sec=10):
    '''
    Guessing the convenient urls
    Arguments:

    '''

    try:
        # Initialiser le driver
        driver = init_driver()
        driver.get(url)
        time.sleep(2)
        
        # Accept pop-up
        accept_popup_general(driver, "#didomi-notice-agree-button")
        time.sleep(5)
        driver.execute_script("document.location.reload()")
        wait = WebDriverWait(driver, wait_sec)

        df_all_versions = pd.DataFrame()
        
        # Sélectionner les options de dropdowns: Marques, Gammes, Annees, Modeles

        # Marque: if contains marque
        time.sleep(2)
        #marque_dropdown = wait.until(EC.presence_of_element_located((By.ID, 'brands')))
        marque_dropdown = driver.find_element(By.ID, 'brands')
        #option_marque =  Select(marque_dropdown).select_by_visible_text(marque)
        option_marque, match_type_marque = select_option_contain (marque_dropdown, marque)
        print(f"Marque: {option_marque}")

        # Gammes: if contains modele
        #modele_dropdown = wait.until(EC.presence_of_element_located((By.ID, 'models')))
        time.sleep(2)
        modele_dropdown = driver.find_element(By.ID, 'models')
        #print(f"Modèle initial est: {modele}")
        option_modele, match_type_modele = select_option_contain (modele_dropdown, modele)
        print(f"Modèle: {option_modele}")
        
        # Annees = annee
        #annee_dropdown = wait.until(EC.presence_of_element_located((By.ID, 'year')))
        time.sleep(2)
        annee_dropdown = driver.find_element(By.ID, 'year')
        option_year, match_type_year = select_option_contain(annee_dropdown, str(annee))
        #print(f"Liste année: {[option.text for option in Select(annee_dropdown).options]}")
        #print(f"Année choisie: {option_year}")
        
        # Modeles: extract all (for i in range modeles.options)
        #modelscomm_dropdown = wait.until(EC.presence_of_element_located((By.ID, 'modelscomm')))
        time.sleep(2)
        modelscomm_dropdown = driver.find_element(By.ID, 'modelscomm')
        select_modelscomm = Select(modelscomm_dropdown)

        # Exclure version vide
        versions = [option.text for option in select_modelscomm.options if option.get_attribute("value").strip() != ""]
        print(f"Liste des versions: {versions}")
       
        if versions:  # This checks if the list is not empty
            for version in versions:
                data = []
                #select_modelscomm.select_by_visible_text(version.text)
                option_marque_cleaned = clean_name(option_marque)
                option_version = clean_name(version)
                
                search_url_version = f"{url}/modele--{option_marque_cleaned}-{option_version}/{option_year}"
                print(f"Extraire la fiche technique de: {version} - année {option_year}")
                print(f"Search url est : {search_url_version}")

                try:
                    driver.get(search_url_version)
                    time.sleep(2)
                    
                    # Récupérer le tableau "listingTab"
                    table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'table.listingTab')))

                    # Get all rows including header row
                    rows = table.find_elements(By.TAG_NAME, 'tr')

                    # Get headers from first row
                    headers = rows[0].find_elements(By.TAG_NAME, 'th')
                    header_texts = [header.text for header in headers]

                    # Get data from remaining rows
                    for row in rows[1:]:
                        cells = row.find_elements(By.TAG_NAME, 'td')
                        row_data = [cell.text for cell in cells]
                        if row_data:
                            data.append(row_data)

                    # Sauvegarder le tableau dans un dataframe
                    df = pd.DataFrame(data, columns=header_texts)

                    # Création des colonnes pour identifier le type de matching
                    df['url'] = driver.current_url
                    df['option_marque_select'] = option_marque
                    df['option_modele_select'] = option_modele
                    df['option_year_select'] = option_year
                    df['match_type_marque'] = match_type_marque
                    df['match_type_modele'] = match_type_modele
                    df['match_type_year'] = match_type_year

                    #print(f"URL used is: {driver.current_url}")

                    #print(df.head())
                    
                    df_all_versions = pd.concat([df_all_versions, df], ignore_index=True)
                    #driver.close()
                    #print(f"df all version est: {df_all_versions.head()}")
                except: # If error then continue with a new iteration
                    continue
        # Close the driver    
        driver.quit() 
        return df_all_versions
    except Exception as e:
        print (f"URL is not correct")
        print (f"Error is: {e}")
        driver.quit()
        return None

In [19]:
# url = "https://www.caradisiac.com/fiches-techniques"
# marque = "BMW"
# modele = "BMW Série 1"
# annee = "2021"
# df_test = collect_prix_neuf (url, marque, modele, annee)

In [20]:
def scraping_prix_neuf (df, csv_path_ok, csv_path_ko, wait_time = 10):
    # Create an empty DataFrame to store all results
    df_all = pd.DataFrame()

    df_error = pd.DataFrame()

    # Add progress tracking
    total_models = len(df)
    print(f"⏳ Starting to scrape {total_models} models & years...")

    # url
    url = "https://www.caradisiac.com/fiches-techniques"

    for idx, row in df.iterrows():
        try:
            print(f"--> 🔄Processing {idx+1}/{total_models}: {row['modele']} ({row['annee']})")
            df_version = collect_prix_neuf (url, row['marque'], row['modele'], row['annee'], wait_time)
            
            if df_version is not None:
                # Add model and year columns to identify the source
                df_version.insert(0, 'source_model', row['modele'])
                df_version.insert(1, 'source_year', row['annee'])
                df_all = pd.concat([df_all, df_version], ignore_index=True)
                print(f"✅Successfully collected for {row['modele']} ({row['annee']})")
            else:
                print(f"❌Error processing {row['modele']} ({row['annee']})")
                df_error = pd.concat([df_error, pd.DataFrame({'modele': [row['modele']], 'annee': [row['annee']]})], ignore_index=True)    
        except Exception as e:
            print(f"❌Error processing {row['modele']} ({row['annee']}): {str(e)}")
            continue

    print(f"\n 🆗 Scraping completed. Total entries collected: {len(df_all)}")
    # Exporter les données dans un fichier CSV
    df_all.to_csv(csv_path_ok, index=False, encoding='utf-8-sig')
    print(f"\n ⚠️ Total models not found: {len(df_error)}")
    df_error.to_csv(csv_path_ko, index=False, encoding='utf-8-sig')
    return None

In [21]:
# Fonction pour diviser le df original en plusieurs parties => pour lancer le scraping en plusieurs patchs
def split_dataframe_by_size(df, nb_lines, csv_root):
    list_dfs = [df[i:i+nb_lines] for i in range(0, len(df), nb_lines)]

    somme_originale = len(df)
    somme_lignes = sum(len(split) for split in list_dfs)

    print(f"Somme des lignes de la base brute = {somme_originale}")
    print(f"Somme des lignes de tous les splits dataframe = {somme_lignes}")
    print(f'# Data split: {len(list_dfs)}')

    # Save each split dataframe to CSV
    for i, split_df in enumerate(list_dfs):
        csv_path = f'{csv_root}/split_car_models_{i+1}.csv'
        split_df.to_csv(csv_path, index=False)
        print(f'Split {i+1} saved to {csv_path}')
    
    return list_dfs

In [22]:
list_df_split = split_dataframe_by_size(df, 50, "../data/raw_data")

Somme des lignes de la base brute = 752
Somme des lignes de tous les splits dataframe = 752
# Data split: 16
Split 1 saved to ../data/raw_data/split_car_models_1.csv
Split 2 saved to ../data/raw_data/split_car_models_2.csv


Split 3 saved to ../data/raw_data/split_car_models_3.csv
Split 4 saved to ../data/raw_data/split_car_models_4.csv
Split 5 saved to ../data/raw_data/split_car_models_5.csv
Split 6 saved to ../data/raw_data/split_car_models_6.csv
Split 7 saved to ../data/raw_data/split_car_models_7.csv
Split 8 saved to ../data/raw_data/split_car_models_8.csv
Split 9 saved to ../data/raw_data/split_car_models_9.csv
Split 10 saved to ../data/raw_data/split_car_models_10.csv
Split 11 saved to ../data/raw_data/split_car_models_11.csv
Split 12 saved to ../data/raw_data/split_car_models_12.csv
Split 13 saved to ../data/raw_data/split_car_models_13.csv
Split 14 saved to ../data/raw_data/split_car_models_14.csv
Split 15 saved to ../data/raw_data/split_car_models_15.csv
Split 16 saved to ../data/raw_data/split_car_models_16.csv


### First attempt

In [23]:
# Process by pack
path = "../data/raw_data"
pattern = "split_car_models_*.csv"
files = glob.glob(os.path.join(path, pattern))
nb_split_df = len(files)

for i, path_split in enumerate(files):
    csv_path_ok = '../data/processed_data/prix_neuf_voitures_pack' + str(i+1) + '.csv'
    
    # Only execute if output file doesn't exist
    if not os.path.exists(csv_path_ok):
        data = pd.read_csv(path_split)
        csv_path_ko = '../data/processed_data/prix_neuf_voitures_pack' + str(i+1) + '_error.csv'

        # Run scraping
        scraping_prix_neuf(data, csv_path_ok, csv_path_ko)
    else:
        print(f"File {csv_path_ok} already exists, skipping...")

File ../data/processed_data/prix_neuf_voitures_pack1.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack2.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack3.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack4.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack5.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack6.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack7.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack8.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack9.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack10.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack11.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack12.csv alrea

In [24]:
# Liste des modèles & années à collecter
df['modele_annee'] = df['modele'] + ' ' + df['annee'].astype(str)
unique_modele_annee = sorted(pd.unique(df["modele_annee"]))
print(f"{len(unique_modele_annee)} modèles et années à collecter: \n {unique_modele_annee}")

# Liste des modèles & années qui n'ont pas été collectés
path = "../data/processed_data"
pattern = "prix_neuf_voitures_pack*.csv"
files = [file for file in glob.glob(os.path.join(path, pattern)) if "error" not in file]
print(files)
df_prix_neuf = pd.DataFrame()
for file in files:
    df_ = pd.read_csv(file)
    df_prix_neuf = pd.concat([df_prix_neuf, df_], ignore_index=True)
df_prix_neuf['modele_annee'] = df_prix_neuf['source_model'] + ' ' + df_prix_neuf['source_year'].astype(str)
unique_modele_annee_collected = sorted(pd.unique(df_prix_neuf["modele_annee"]))
print(f"{len(unique_modele_annee_collected)} modèles et années collectés: \n {unique_modele_annee_collected}")

# Liste des modèles & années qui ont été collectés (sans doublons)
df_prix_neuf['modele_annee_option'] = df_prix_neuf['option_marque_select'] +  ' ' + df_prix_neuf['option_modele_select'] + ' ' + df_prix_neuf['option_year_select'].astype(str)
unique_modele_annee_option = sorted(pd.unique(df_prix_neuf["modele_annee_option"]))
print(f"{len(unique_modele_annee_option)} modèles et années sélectionnés: \n {unique_modele_annee_option}")

unique_modele_annee_final = df_prix_neuf[['source_model', 'source_year', 'modele_annee_option']].drop_duplicates()
unique_modele_annee_final['duplicate_source_models'] = unique_modele_annee_final.groupby('modele_annee_option')['source_model'].transform(lambda x: '#'.join(x) if len(x) > 1 else None)
df_doublons = unique_modele_annee_final[['modele_annee_option', 'duplicate_source_models']].drop_duplicates()
df_doublons = df_doublons[df_doublons['duplicate_source_models'].notnull()]
df_doublons

752 modèles et années à collecter: 
 ['Abarth 124 Spider 2019', 'Abarth 500 2018', 'Abarth 500 2019', 'Abarth 500 2020', 'Abarth 500 2022', 'Abarth 595 2017', 'Abarth 595C 2020', 'Alfa Romeo Giulia 2020', 'Alfa Romeo Giulia 2021', 'Alfa Romeo Giulietta 2017', 'Alfa Romeo Giulietta 2018', 'Alfa Romeo Giulietta 2019', 'Alfa Romeo Giulietta 2021', 'Alfa Romeo Stelvio 2017', 'Alfa Romeo Stelvio 2018', 'Alfa Romeo Tonale 2023', 'Alfa Romeo Tonale 2024', 'Audi A1 2017', 'Audi A1 2018', 'Audi A1 2019', 'Audi A1 2020', 'Audi A1 2021', 'Audi A1 2023', 'Audi A1 citycarver 2021', 'Audi A3 2017', 'Audi A3 2018', 'Audi A3 2019', 'Audi A3 2020', 'Audi A3 2021', 'Audi A3 2022', 'Audi A3 2023', 'Audi A3 Limousine 2017', 'Audi A3 Limousine 2019', 'Audi A3 Limousine 2021', 'Audi A4 2018', 'Audi A4 2019', 'Audi A4 2020', 'Audi A4 2021', 'Audi A4 2022', 'Audi A5 Sportback 2020', 'Audi A5 Sportback 2022', 'Audi A6 2018', 'Audi Q2 2017', 'Audi Q2 2018', 'Audi Q2 2019', 'Audi Q2 2020', 'Audi Q2 2022', 'Audi 

,modele_annee_option,duplicate_source_models
1393,AUDI A1 2021,Audi A1#Audi A1 citycarver
1497,AUDI A3 2017,Audi A3#Audi A3 Limousine
2462,AUDI A3 2019,Audi A3#Audi A3 Limousine
2737,AUDI A3 2021,Audi A3#Audi A3 Limousine
8787,FIAT 500 2017,Fiat 500#Fiat 500C#Fiat 500X
8981,FIAT 500 2018,Fiat 500#Fiat 500C#Fiat 500X
9194,FIAT 500 2019,Fiat 500#Fiat 500C#Fiat 500X
16198,HYUNDAI I30 2019,Hyundai i30#Hyundai i30 Fastback
23573,MERCEDES CLASSE A 2019,Mercedes-Benz Classe A#Mercedes-Benz Classe A ...
23668,MERCEDES CLASSE A 2020,Mercedes-Benz Classe A#Mercedes-Benz Classe A ...


### Second attempt

In [25]:
# Liste des modèles & années restant à collecter
remaining_to_collect = set(unique_modele_annee) - set(unique_modele_annee_collected)
print(len(remaining_to_collect))
print(sorted(remaining_to_collect))

102
['Abarth 595 2017', 'Abarth 595C 2020', 'Audi A1 2023', 'Audi A3 2020', 'BMW Série 4 2019', 'BMW Série 4 2022', 'Citroen C4 Grand Picasso 2017', 'Citroen C4 Grand Picasso 2019', 'Citroen C4 Picasso 2019', 'Citroen DS3 2018', 'Citroen DS3 Cabrio 2017', 'Citroen DS3 Cabrio 2019', 'Citroen DS4 2018', 'Citroen DS4 Crossback 2018', 'Citroen DS5 2017', 'Citroen DS5 2018', 'DS Automobiles DS3 2019', 'DS Automobiles DS3 2022', 'DS Automobiles DS3 2023', 'DS Automobiles DS3 Crossback 2019', 'DS Automobiles DS3 Crossback 2020', 'DS Automobiles DS3 Crossback 2021', 'DS Automobiles DS3 Crossback 2022', 'DS Automobiles DS4 2021', 'DS Automobiles DS4 2022', 'DS Automobiles DS7 2022', 'DS Automobiles DS7 Crossback 2018', 'DS Automobiles DS7 Crossback 2020', 'DS Automobiles DS7 Crossback 2021', 'Dacia Dokker 2019', 'Dacia Duster 2018', 'Fiat 500 2020', 'Fiat 500 2021', 'Fiat 500 2022', 'Fiat 500C 2020', 'Fiat 500C 2021', 'Fiat 500L 2020', 'Fiat 500X 2020', 'Fiat 500X 2021', 'Ford EcoSport 2021', '

In [26]:
# Dataframe des modèles restant à collecter
df2 = df[df['modele_annee'].isin(remaining_to_collect)]
df2 = df2.reset_index(drop=True)

In [27]:
df2.head()

,marque,modele,annee,modele_annee
0,Abarth,Abarth 595,2017,Abarth 595 2017
1,Abarth,Abarth 595C,2020,Abarth 595C 2020
2,Audi,Audi A1,2023,Audi A1 2023
3,Audi,Audi A3,2020,Audi A3 2020
4,BMW,BMW Série 4,2019,BMW Série 4 2019


In [28]:
# Il y avait peut-être des erreurs lors du 1er scraping à cause du serveur trop chargé
# 2è essaie...
csv_path_ok = '../data/processed_data/prix_neuf_voitures_essaie_2.csv'
if not os.path.exists(csv_path_ok):
    csv_path_ko = '../data/processed_data/prix_neuf_voitures_essaie_2_error.csv'
    scraping_prix_neuf (df2, csv_path_ok, csv_path_ko)
else:
    print(f"File {csv_path_ok} already exists, skipping...")

File ../data/processed_data/prix_neuf_voitures_essaie_2.csv already exists, skipping...


In [29]:
# Importer les résultats
df2_ok = pd.read_csv('../data/processed_data/prix_neuf_voitures_essaie_2.csv')
df2_ok.head()

,source_model,source_year,Versions,Portes,Energie,Boite,CO2\n(g/km),Prix,url,option_marque_select,option_modele_select,option_year_select,match_type_marque,match_type_modele,match_type_year
0,Audi A1,2023,II ALLSTREET 25 TFSI 95 ADVANCED BVM5,5.0,Ess.,Mécanique,130 (wltp),27 040 €,https://www.caradisiac.com/fiches-techniques/m...,AUDI,A1,2023,Exact match,Partial match,Exact match
1,Audi A1,2023,II ALLSTREET 25 TFSI 95 DESIGN BVM5,5.0,Ess.,Mécanique,129 (wltp),26 510 €,https://www.caradisiac.com/fiches-techniques/m...,AUDI,A1,2023,Exact match,Partial match,Exact match
2,Audi A1,2023,II ALLSTREET 30 TFSI 110 ADVANCED BVM6,5.0,Ess.,Mécanique,128 (wltp),27 750 €,https://www.caradisiac.com/fiches-techniques/m...,AUDI,A1,2023,Exact match,Partial match,Exact match
3,Audi A1,2023,II ALLSTREET 30 TFSI 110 ADVANCED S TRONIC 7,5.0,Ess.,Automatique,141 (wltp),29 650 €,https://www.caradisiac.com/fiches-techniques/m...,AUDI,A1,2023,Exact match,Partial match,Exact match
4,Audi A1,2023,II ALLSTREET 30 TFSI 110 AVUS BVM6,5.0,Ess.,Mécanique,128 (wltp),33 200 €,https://www.caradisiac.com/fiches-techniques/m...,AUDI,A1,2023,Exact match,Partial match,Exact match


In [30]:
# Liste des modèles qui sont collectés avec la 2è essaie
df2_ok['modele_annee'] = df2_ok['source_model'] + ' ' + df2_ok['source_year'].astype(str)
unique_modele_annee_collected_sup = sorted(pd.unique(df2_ok["modele_annee"]))
print(f"{len(unique_modele_annee_collected_sup)} modèles et années collectés avec la 2è essaie: \n {unique_modele_annee_collected_sup}")

# Liste des modèles qui ne sont toujours pas collectés avec la 2è essaie
remaining_to_collect_sup = sorted(set(remaining_to_collect) - set(unique_modele_annee_collected_sup))
print(f"{len(remaining_to_collect_sup)} modèles et années restant à collecter: \n {remaining_to_collect_sup}")

38 modèles et années collectés avec la 2è essaie: 
 ['Audi A1 2023', 'DS Automobiles DS3 2019', 'DS Automobiles DS3 2022', 'DS Automobiles DS3 2023', 'DS Automobiles DS3 Crossback 2020', 'DS Automobiles DS3 Crossback 2021', 'DS Automobiles DS3 Crossback 2022', 'DS Automobiles DS4 2021', 'DS Automobiles DS4 2022', 'DS Automobiles DS7 2022', 'DS Automobiles DS7 Crossback 2018', 'DS Automobiles DS7 Crossback 2020', 'DS Automobiles DS7 Crossback 2021', 'Dacia Dokker 2019', 'Dacia Duster 2018', 'Fiat 500L 2020', 'Fiat 500X 2020', 'Fiat 500X 2021', 'Mazda 3 2022', 'Mazda MX-5 2017', 'Mazda MX-5 2019', 'Peugeot 2008 2018', 'Renault Twingo 2017', 'Renault Twingo 2018', 'Seat Ibiza 2021', 'Seat Leon 2019', 'Skoda Kodiaq 2018', 'Skoda Kodiaq 2020', 'Skoda Kodiaq 2021', 'Toyota Auris 2017', 'Toyota Aygo 2018', 'Toyota Aygo 2019', 'Toyota Aygo X 2022', 'Toyota Aygo X 2024', 'Toyota C-HR 2021', 'Toyota Yaris 2024', 'Toyota Yaris Cross 2021', 'Toyota Yaris Cross 2022']
64 modèles et années restant à

### Third attempt

In [31]:
df3 = df[df['modele_annee'].isin(remaining_to_collect_sup)]
df3 = df3.reset_index(drop=True)
df3

,marque,modele,annee,modele_annee
0,Abarth,Abarth 595,2017,Abarth 595 2017
1,Abarth,Abarth 595C,2020,Abarth 595C 2020
2,Audi,Audi A3,2020,Audi A3 2020
3,BMW,BMW Série 4,2019,BMW Série 4 2019
4,BMW,BMW Série 4,2022,BMW Série 4 2022
...,...,...,...,...
59,Toyota,Toyota Yaris,2017,Toyota Yaris 2017
60,Toyota,Toyota Yaris,2018,Toyota Yaris 2018
61,Toyota,Toyota Yaris,2019,Toyota Yaris 2019
62,Toyota,Toyota Yaris,2020,Toyota Yaris 2020


In [32]:
# 3è essaie avec wait time plus élevé
csv_path_ok = '../data/processed_data/prix_neuf_voitures_essaie_3.csv'
if not os.path.exists(csv_path_ok):
    csv_path_ko = '../data/processed_data/prix_neuf_voitures_essaie_3_error.csv'
    scraping_prix_neuf (df3, csv_path_ok, csv_path_ko, 20)
else:
    print(f"File {csv_path_ok} already exists, skipping...")

File ../data/processed_data/prix_neuf_voitures_essaie_3.csv already exists, skipping...


### Fourth attempt

In [33]:
df3_ok = pd.read_csv('../data/processed_data/prix_neuf_voitures_essaie_3.csv')
# Liste des modèles qui sont collectés avec la 3è essaie
df3_ok['modele_annee'] = df3_ok['source_model'] + ' ' + df3_ok['source_year'].astype(str)
unique_modele_annee_collected_sup_2 = sorted(pd.unique(df3_ok["modele_annee"]))
print(f"{len(unique_modele_annee_collected_sup_2)} modèles et années collectés avec la 3è essaie: \n {unique_modele_annee_collected_sup_2}")

# Liste des modèles qui ne sont toujours pas collectés avec la 3è essaie
remaining_to_collect_sup_2 = sorted(set(remaining_to_collect_sup) - set(unique_modele_annee_collected_sup_2))
print(f"{len(remaining_to_collect_sup_2)} modèles et années restant à collecter: \n {remaining_to_collect_sup_2}")

27 modèles et années collectés avec la 3è essaie: 
 ['Audi A3 2020', 'BMW Série 4 2019', 'BMW Série 4 2022', 'DS Automobiles DS3 Crossback 2019', 'Fiat 500 2020', 'Fiat 500 2021', 'Fiat 500 2022', 'Fiat 500C 2020', 'Fiat 500C 2021', 'Ford EcoSport 2021', 'Ford EcoSport 2022', 'Mazda 6 2020', 'Opel Astra 2018', 'Opel Corsa 2017', 'Opel Corsa 2018', 'Opel Corsa 2019', 'Renault Mégane 2017', 'Renault Mégane 2018', 'Renault Mégane 2019', 'Renault Mégane 2020', 'Renault Mégane 2021', 'Renault Mégane 2022', 'Toyota Yaris 2017', 'Toyota Yaris 2018', 'Toyota Yaris 2019', 'Toyota Yaris 2020', 'Volkswagen Caddy 2019']
37 modèles et années restant à collecter: 
 ['Abarth 595 2017', 'Abarth 595C 2020', 'Citroen C4 Grand Picasso 2017', 'Citroen C4 Grand Picasso 2019', 'Citroen C4 Picasso 2019', 'Citroen DS3 2018', 'Citroen DS3 Cabrio 2017', 'Citroen DS3 Cabrio 2019', 'Citroen DS4 2018', 'Citroen DS4 Crossback 2018', 'Citroen DS5 2017', 'Citroen DS5 2018', 'Ford Ka+ 2018', 'Ford Tourneo 2020', "Kia 

In [49]:
df4 = df3[df3['modele_annee'].isin(remaining_to_collect_sup_2)].reset_index(drop=True)
# liste des modèles
list_modele_df4 = pd.unique(df4['modele'])
print(list_modele_df4)

['Abarth 595' 'Abarth 595C' 'Citroen C4 Grand Picasso'
 'Citroen C4 Picasso' 'Citroen DS3' 'Citroen DS3 Cabrio' 'Citroen DS4'
 'Citroen DS4 Crossback' 'Citroen DS5' 'Ford Ka+' 'Ford Tourneo'
 "Kia cee'd" "Kia pro_cee'd" 'Land Rover Evoque'
 'Mercedes-Benz Classe GLB' 'Mercedes-Benz Classe GLC'
 'Mercedes-Benz Classe GLE' 'Opel Crossland X' 'Suzuki SX4 S-Cross']


In [50]:
# Corriger manuellement le nom du modèle/marque pour qu'il soit aligné avec celui utilisé par le site
dict_model_corr = {

 'Abarth 595' : '500',
 'Abarth 595C' : '500',
 'Citroen C4 Grand Picasso' : 'C4 PICASSO',
 'Citroen C4 Picasso' : 'C4 PICASSO', 
 'Citroen DS3' : 'DS3',
 'Citroen DS3 Cabrio' : 'DS3',
 'Citroen DS4' : 'DS4',
 'Citroen DS4 Crossback' : 'DS4 CROSSBACK',
 'Citroen DS5' : 'DS5',
 'Ford Ka+' : 'KA+',
 'Ford Tourneo' : 'TOURNEO COURIER',
 "Kia cee'd" : 'CEED',
 "Kia pro_cee'd": 'PROCEED',
 'Land Rover Evoque' : 'RANGE ROVER EVOQUE',
 'Mercedes-Benz Classe GLB' : 'GLB',
 'Mercedes-Benz Classe GLC' : 'GLC',
 'Mercedes-Benz Classe GLE' : 'GLE',
 'Opel Crossland X' : 'CROSSLAND' ,
 'Suzuki SX4 S-Cross' : 'S-CROSS'
}

In [51]:
# Create columns to keep original "marque" & "modele" name
df4["marque_init"] = df4["marque"]
df4['modele_init'] = df4['modele']

# Replace the "modele" name
df4['modele'] = df4['modele'].replace(dict_model_corr)
df4.head()

,marque,modele,annee,modele_annee,marque_init,modele_init
0,Abarth,500,2017,Abarth 595 2017,Abarth,Abarth 595
1,Abarth,500,2020,Abarth 595C 2020,Abarth,Abarth 595C
2,Citroen,C4 PICASSO,2017,Citroen C4 Grand Picasso 2017,Citroen,Citroen C4 Grand Picasso
3,Citroen,C4 PICASSO,2019,Citroen C4 Grand Picasso 2019,Citroen,Citroen C4 Grand Picasso
4,Citroen,C4 PICASSO,2019,Citroen C4 Picasso 2019,Citroen,Citroen C4 Picasso


In [53]:
#df4[(df4["modele"].str.contains("DS", na=False)) & (df4["marque"].str.contains("Citroen", na=False))]

In [54]:
# Rename "marque" Citroen = DS if "modele" contains DS and "marque" contains Citroen
df4.loc[(df4["modele"].str.contains("DS", na=False)) 
        & (df4["marque"].str.contains("Citroen", na=False)), "marque"] = "DS"
df4['modele_annee_upd'] = df4['modele']+ ' ' + df4['annee'].astype(str)
list_to_collect_4th = sorted(pd.unique(df4['modele_annee_upd']))
print(f"{len(list_to_collect_4th)} à collecter: \n {list_to_collect_4th}")

36 à collecter: 
 ['500 2017', '500 2020', 'C4 PICASSO 2017', 'C4 PICASSO 2019', 'CEED 2018', 'CEED 2019', 'CEED 2021', 'CEED 2022', 'CEED 2023', 'CROSSLAND 2023', 'DS3 2017', 'DS3 2018', 'DS3 2019', 'DS4 2018', 'DS4 CROSSBACK 2018', 'DS5 2017', 'DS5 2018', 'GLB 2020', 'GLB 2021', 'GLB 2022', 'GLC 2017', 'GLC 2018', 'GLC 2019', 'GLC 2020', 'GLC 2021', 'GLE 2018', 'KA+ 2018', 'PROCEED 2017', 'PROCEED 2019', 'PROCEED 2020', 'PROCEED 2021', 'RANGE ROVER EVOQUE 2017', 'RANGE ROVER EVOQUE 2018', 'RANGE ROVER EVOQUE 2020', 'S-CROSS 2021', 'TOURNEO COURIER 2020']


In [38]:
# url = "https://www.caradisiac.com/fiches-techniques"
# marque = "KIA"
# modele = "PROCEED"
# annee = "2021"
# df_test = collect_prix_neuf (url, marque, modele, annee)

In [39]:
# 4è essaie
csv_path_ok = '../data/processed_data/prix_neuf_voitures_essaie_4.csv'
if not os.path.exists(csv_path_ok):
    csv_path_ko = '../data/processed_data/prix_neuf_voitures_essaie_4_error.csv'
    scraping_prix_neuf (df4, csv_path_ok, csv_path_ko, 10)
else:
    print(f"File {csv_path_ok} already exists, skipping...")

File ../data/processed_data/prix_neuf_voitures_essaie_4.csv already exists, skipping...


In [40]:
# Import the result from 4th attempt
df4_ok = pd.read_csv(csv_path_ok)
df4_ok

# Liste des modèles qui sont collectés avec la 4è essaie
df4_ok['modele_annee'] = df4_ok['source_model'] + ' ' + df4_ok['source_year'].astype(str)
unique_modele_annee_collected_sup_3 = sorted(pd.unique(df4_ok["modele_annee"]))
print(f"{len(unique_modele_annee_collected_sup_3)} modèles et années collectés avec la 4è essaie: \n {unique_modele_annee_collected_sup_3}")

# Liste des modèles qui ne sont toujours pas collectés avec la 3è essaie
remaining_to_collect_sup_3 = sorted(set(list_to_collect_4th) - set(unique_modele_annee_collected_sup_3))
print(f"{len(remaining_to_collect_sup_3)} modèles et années restant à collecter: \n {remaining_to_collect_sup_3}")

34 modèles et années collectés avec la 4è essaie: 
 ['500 2017', '500 2020', 'C4 PICASSO 2017', 'CEED 2018', 'CEED 2019', 'CEED 2021', 'CEED 2022', 'CEED 2023', 'CROSSLAND 2023', 'DS3 2017', 'DS3 2018', 'DS3 2019', 'DS4 2018', 'DS4 CROSSBACK 2018', 'DS5 2017', 'DS5 2018', 'GLB 2020', 'GLB 2021', 'GLB 2022', 'GLC 2017', 'GLC 2018', 'GLC 2019', 'GLC 2020', 'GLC 2021', 'GLE 2018', 'KA+ 2018', 'PROCEED 2019', 'PROCEED 2020', 'PROCEED 2021', 'RANGE ROVER EVOQUE 2017', 'RANGE ROVER EVOQUE 2018', 'RANGE ROVER EVOQUE 2020', 'S-CROSS 2021', 'TOURNEO COURIER 2020']
2 modèles et années restant à collecter: 
 ['C4 PICASSO 2019', 'PROCEED 2017']


### Final

In [ ]:
# Concatenate the dataframes
df_combined = pd.concat([df_prix_neuf, df2_ok, df3_ok, df4_ok], ignore_index=True)

# Display the combined dataframe
df_combined.head()

# Remove duplicates
df_combined_nodup = df_combined.drop_duplicates(subset=['option_marque_select', 'option_modele_select', 'option_year_select', 'Versions'])
list_final = sorted(pd.unique(df_combined_nodup["modele_annee"]))

print(len(list_final))

724


In [42]:
df_combined_nodup.head()

,source_model,source_year,Versions,Portes,Energie,Boite,CO2\n(g/km),Prix,url,option_marque_select,option_modele_select,option_year_select,match_type_marque,match_type_modele,match_type_year,modele_annee,modele_annee_option
0,Abarth 124 Spider,2019,II 1.4 TURBO 170,2.0,Ess.,Mécanique,146 (nedc),34 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
1,Abarth 124 Spider,2019,II 1.4 TURBO 170 GT,2.0,Ess.,Mécanique,146 (nedc),40 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
2,Abarth 124 Spider,2019,II 1.4 TURBO 170 GT BVA,2.0,Ess.,Automatique,161 (nedc),42 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
3,Abarth 124 Spider,2019,II 1.4 TURBO 170 TURISMO,2.0,Ess.,Mécanique,146 (nedc),37 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
4,Abarth 124 Spider,2019,II 1.4 TURBO 170 TURISMO BVA,2.0,Ess.,Automatique,161 (nedc),39 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019


## 4. Merging with initial data to get the listing price

In [45]:
df_model[(df_model['modele'].str.lower().str.contains("ds", na=False)) & (df_model['marque'].str.lower().str.contains('citroen', na=False))]

,marque,modele,finition,annee
273,Citroen,Citroen DS3,1.2 PureTech Connected Chic EAT6,2018
274,Citroen,Citroen DS3,1.2 PureTech Performance Line,2018
275,Citroen,Citroen DS3 Cabrio,Cabriolet 1.2 PureTech Performance Line Automa...,2019
276,Citroen,Citroen DS3 Cabrio,Cabriolet 1.2 PureTech So Chic EAT6,2017
277,Citroen,Citroen DS4,1.2 PureTech Be Chic BV6,2018
278,Citroen,Citroen DS4 Crossback,1.2 PureTech Sport Chic,2018
279,Citroen,Citroen DS5,2.0 Blue-HDi Sport Chic EAT6,2017
280,Citroen,Citroen DS5,2.0 Blue-HDi Sport Chic EAT6,2018


In [ ]:
# Rename "marque" Citroen = DS if "modele" contains DS and "marque" contains Citroen
df_model.loc[(df_model['marque'].str.lower().str.contains('citroen', na=False)) & (df_model['modele'].str.lower().str.contains('ds', na=False)), "marque"] = "DS"

In [ ]:
# Checking again, should be empty
df_model[(df_model['modele'].str.lower().str.contains("ds", na=False)) & (df_model['marque'].str.lower().str.contains('citroen', na=False))]

,marque,modele,finition,annee


In [ ]:
df_model

,marque,modele,finition,annee
0,Abarth,Abarth 124 Spider,1.4 Turbo Turismo BVA,2019
1,Abarth,Abarth 500,1.4 Turbo T-Jet 595,2020
2,Abarth,Abarth 500,1.4 Turbo T-Jet 595,2022
3,Abarth,Abarth 500,1.4 Turbo T-Jet 595 Pista,2018
4,Abarth,Abarth 500,1.4 Turbo T-Jet 595 Turismo,2020
...,...,...,...,...
1608,Volvo,Volvo V40,2.0 D2 Inscription,2018
1609,Volvo,Volvo V40 Cross Country,Cross Country 1.5 T3 Momentum Geartronic 6,2019
1610,Volvo,Volvo XC40,1.5 T3 R-Design Geartronic 8,2019
1611,Volvo,Volvo XC40,2.0 D4 AWD AdBlue R-Design Geartronic 8,2019


In [61]:
df_combined_nodup

,source_model,source_year,Versions,Portes,Energie,Boite,CO2\n(g/km),Prix,url,option_marque_select,option_modele_select,option_year_select,match_type_marque,match_type_modele,match_type_year,modele_annee,modele_annee_option
0,Abarth 124 Spider,2019,II 1.4 TURBO 170,2.0,Ess.,Mécanique,146 (nedc),34 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
1,Abarth 124 Spider,2019,II 1.4 TURBO 170 GT,2.0,Ess.,Mécanique,146 (nedc),40 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
2,Abarth 124 Spider,2019,II 1.4 TURBO 170 GT BVA,2.0,Ess.,Automatique,161 (nedc),42 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
3,Abarth 124 Spider,2019,II 1.4 TURBO 170 TURISMO,2.0,Ess.,Mécanique,146 (nedc),37 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
4,Abarth 124 Spider,2019,II 1.4 TURBO 170 TURISMO BVA,2.0,Ess.,Automatique,161 (nedc),39 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62032,S-CROSS,2021,1.4 BOOSTERJET HYBRID STYLE,5.0,Ess.,Mécanique,122 (wltp),32 240 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,S-CROSS,2021,Very Exact match,Very Exact match,Very Exact match,S-CROSS 2021,NaN
62033,S-CROSS,2021,1.4 BOOSTERJET HYBRID STYLE 2020,5.0,Ess.,Mécanique,123 (wltp),27 640 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,S-CROSS,2021,Very Exact match,Very Exact match,Very Exact match,S-CROSS 2021,NaN
62034,S-CROSS,2021,1.4 BOOSTERJET HYBRID STYLE ALLGRIP,5.0,Ess.,Mécanique,133 (wltp),29 640 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,S-CROSS,2021,Very Exact match,Very Exact match,Very Exact match,S-CROSS 2021,NaN
62035,S-CROSS,2021,1.4 BOOSTERJET HYBRID STYLE ALLGRIP AUTO,5.0,Ess.,Automatique,142 (wltp),31 340 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,S-CROSS,2021,Very Exact match,Very Exact match,Very Exact match,S-CROSS 2021,NaN


In [65]:
# Remove missing price
df_combined_nodup = df_combined_nodup[df_combined_nodup["Prix"].notna()]
df_combined_nodup['Prix'].describe()

count        52423
unique        5829
top       28 900 €
freq           129
Name: Prix, dtype: object

In [ ]:
# Checking missing price is well removed
missing_price = df_combined_nodup['Prix'].isnull().sum()
print(missing_price)

0


In [70]:
df_combined_nodup

,source_model,source_year,Versions,Portes,Energie,Boite,CO2\n(g/km),Prix,url,option_marque_select,option_modele_select,option_year_select,match_type_marque,match_type_modele,match_type_year,modele_annee,modele_annee_option
0,Abarth 124 Spider,2019,II 1.4 TURBO 170,2.0,Ess.,Mécanique,146 (nedc),34 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
1,Abarth 124 Spider,2019,II 1.4 TURBO 170 GT,2.0,Ess.,Mécanique,146 (nedc),40 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
2,Abarth 124 Spider,2019,II 1.4 TURBO 170 GT BVA,2.0,Ess.,Automatique,161 (nedc),42 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
3,Abarth 124 Spider,2019,II 1.4 TURBO 170 TURISMO,2.0,Ess.,Mécanique,146 (nedc),37 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
4,Abarth 124 Spider,2019,II 1.4 TURBO 170 TURISMO BVA,2.0,Ess.,Automatique,161 (nedc),39 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62031,S-CROSS,2021,1.4 BOOSTERJET HYBRID PRIVILEGE AUTO,5.0,Ess.,Automatique,129 (wltp),27 990 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,S-CROSS,2021,Very Exact match,Very Exact match,Very Exact match,S-CROSS 2021,NaN
62032,S-CROSS,2021,1.4 BOOSTERJET HYBRID STYLE,5.0,Ess.,Mécanique,122 (wltp),32 240 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,S-CROSS,2021,Very Exact match,Very Exact match,Very Exact match,S-CROSS 2021,NaN
62033,S-CROSS,2021,1.4 BOOSTERJET HYBRID STYLE 2020,5.0,Ess.,Mécanique,123 (wltp),27 640 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,S-CROSS,2021,Very Exact match,Very Exact match,Very Exact match,S-CROSS 2021,NaN
62034,S-CROSS,2021,1.4 BOOSTERJET HYBRID STYLE ALLGRIP,5.0,Ess.,Mécanique,133 (wltp),29 640 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,S-CROSS,2021,Very Exact match,Very Exact match,Very Exact match,S-CROSS 2021,NaN


In [ ]:
# Merging df_model with df_combined_nodup to get the listing price
df_model['marque'] = df_model['marque'].str.upper()
df_model['finition'] = df_model['finition'].str.upper()

# 1. Exact match
df_test = df_model.merge(df_combined_nodup, how="left", left_on=['marque', 'finition', 'annee'], right_on = ['option_marque_select', 'Versions', "option_year_select"])

# 2. Partial match

In [76]:
df_test

,marque,modele,finition,annee,source_model,source_year,Versions,Portes,Energie,Boite,...,Prix,url,option_marque_select,option_modele_select,option_year_select,match_type_marque,match_type_modele,match_type_year,modele_annee,modele_annee_option
0,ABARTH,Abarth 124 Spider,1.4 TURBO TURISMO BVA,2019,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABARTH,Abarth 500,1.4 TURBO T-JET 595,2020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABARTH,Abarth 500,1.4 TURBO T-JET 595,2022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABARTH,Abarth 500,1.4 TURBO T-JET 595 PISTA,2018,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABARTH,Abarth 500,1.4 TURBO T-JET 595 TURISMO,2020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608,VOLVO,Volvo V40,2.0 D2 INSCRIPTION,2018,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1609,VOLVO,Volvo V40 Cross Country,CROSS COUNTRY 1.5 T3 MOMENTUM GEARTRONIC 6,2019,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1610,VOLVO,Volvo XC40,1.5 T3 R-DESIGN GEARTRONIC 8,2019,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1611,VOLVO,Volvo XC40,2.0 D4 AWD ADBLUE R-DESIGN GEARTRONIC 8,2019,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
